In [ ]:
import os
import csv
import pandas as pd
import numpy as np

import datetime
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()

%matplotlib inline

from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import squarify

from sklearn import model_selection, preprocessing, metrics
plt.style.use('fivethirtyeight')

print(os.getcwd())
print(os.listdir("../"))

In [ ]:
def clean_data(df):
    #since first column contains row number
#     df.drop(["Row_num"], axis=1)
    #removing  and kesEntityId as simillar fields are already present
    df = df.drop(columns=["Row_num","kesEntityId"],axis=1)
    df["Popularity"] = df["Popularity"].apply(pd.to_numeric,downcast='float')
    df["Year"] = df["Year"].apply(pd.to_numeric,downcast='unsigned')
    df["Month"] = df["Month"].apply(pd.to_numeric,downcast='unsigned')
    df = df.dropna(subset=['Venue'])
    return df

In [ ]:
df_pub = pd.read_csv("../input/scopus-data/ProcessedScopusData.csv")

In [ ]:
def append_pub_score(df):
    print("Unique Publications from Scopus:",len(df_pub.Title.unique()))
    # Analysing Common Publications
    vfn_list = df.Venue.unique().tolist()
    scopus_list = df_pub.Title.unique().tolist()
    count = 0
    for pub in scopus_list:
        if pub in vfn_list:
            count +=1
    print("Publications present in Dataset:",count)
    print("Shape Before",df.shape)
    df = df[df.Venue.isin(scopus_list)]
    print("Shape After",df.shape)
    #score appending
    score_dict = pd.Series(df_pub.SJR.values,index=df_pub.Title).to_dict()
    df["Publication_Rank"] = df.apply(lambda row: score_dict[row["Venue"]],axis = 1)
    return df

def extract_field(row):
    val = row["Domain"]
    index = val.rfind("FN")
    val = val[index+3:len(val)]
    val = val.strip(":}] '")
    return val

def get_num_authors(row):
    vals = row["Authors"].split("},")
    return len(vals)

In [ ]:
label_dict = {"C":0,"J":1,"CJ":2,"O":3}
def find_conference_type(row):
    if pd.isnull(row["Conference"]) and pd.isnull(row["Journal"]):
        return label_dict["O"]
    elif pd.isnull(row["Conference"]):
        return label_dict["J"]
    elif pd.isnull(row["Journal"]):
        return label_dict["C"]
    else:
        return label_dict["CJ"]

In [ ]:
def tidy_split(df, column, sep='|', keep=False):
    indexes = list()
    id_values = list()
    name_values = list()
    df = df.dropna(subset=[column])
    for i, presplit in enumerate(df[column].astype(str)):
        values = presplit.split(sep)
        if keep and len(values) > 1:
            indexes.append(i)
            id_values.append(presplit)
        for value in values:
            indexes.append(i)
            val = value
            index1 = val.find("AuId")
            index2 = val.find("AfN")
            index3 = val.find("'S'")
            if index3 < index1:
                val = val[index1+4:index2]
                val = val.strip("', :")
            else:
                val = val[index1+4:index3]
                val = val.strip("', :")                
            id_values.append(val)
            index1 = value.find("AuN")
            val = value[index1+3:]
            val = val.strip("': }]")
            name_values.append(val)
    new_df = df.iloc[indexes, :].copy()
    new_df[column+"_Id"] = id_values
    new_new_df = new_df.iloc[:, :].copy()
    new_new_df[column+"_Name"] = name_values
    return new_new_df

In [ ]:
def process_year(df):
    # Make more sensible column names
    rename_dict = {"AA": "Authors", 
                   "C": "Conference", 
                   "CC": "Citation_count",
                   "ECC": "Expected_count",
                   "D": "Date", 
                   "F": "Domain", 
                   "J": "Journal", 
                   "RId": "References", 
                   "Ti": "Title", 
                   "VFN": "Venue", 
                   "W": "Words", 
                   "Y": "Year", 
                   "logprob":"Popularity",
                   "Unnamed: 0": "Row_num"}
    df.rename(index=str, columns=rename_dict, inplace=True)
    df.drop_duplicates(subset=['Title'], inplace=True)
    df["Month"] = pd.to_datetime(df["Date"]).dt.month
    df = clean_data(df)
    df = append_pub_score(df)
    df["Publication Type"] = df.apply(lambda row:find_conference_type(row),axis=1)
    df["Number_Of_authors"] = df.apply(lambda row:get_num_authors(row), axis = 1)
    return df

In [ ]:
df = pd.read_csv("../input/mag-2007/mag_2007.csv")

In [ ]:
# Preprocessed files are saved in drive, we do not need to process it. Download those files and read them
# df = process_year(df)

In [ ]:
# df.to_csv("mag_2007_processed.csv")